In [2]:
%load_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

# Задание 1

Вывести 10 самых больших по размеру треков жанра ROCK и формата MPEG (используйте limit)

In [25]:
%%sql
select t.name from tracks t
left join media_types m on t.MediaTypeId = m.MediaTypeId
left join genres g on g.GenreId = t.GenreId
where m.Name like '%mpeg%' and g.Name like 'ROCK'
order by t.Bytes desc
limit 10


 * sqlite:///chinook.db
Done.


Name
Dazed And Confused
Space Truckin'
Dazed And Confused
We've Got To Get Together/Jingo
Funky Piano
Going Down / Highway Star
Santana Jam
The Sun Road
Whole Lotta Love
Mistreated (Alternate Version)


## Задание 2
Вывести названия всех групп, их песен и названия их альбомов для всех треков жанра Рок, приобретенные сотрудниками Microsoft.

In [32]:
%%sql
select artists.name as "Artist", tracks.Name as "Track", Title as "Album" 
from tracks
left join albums on tracks.AlbumId = albums.AlbumId
left join artists on artists.ArtistId = albums.ArtistId
left join genres on tracks.GenreId = genres.GenreId
left join invoice_items on invoice_items.TrackId = tracks.TrackId
left join invoices on invoices.InvoiceId = invoice_items.InvoiceId
left join customers on customers.CustomerId = invoices.CustomerId
where genres.Name = "Rock" and customers.Company like 'Microsoft%'


 * sqlite:///chinook.db
Done.


Artist,Track,Album
Foo Fighters,No Way Back,In Your Honor [Disc 1]
Foo Fighters,Still,In Your Honor [Disc 2]
Foo Fighters,Razor,In Your Honor [Disc 2]
Foo Fighters,My Hero,The Colour And The Shape
Guns N' Roses,Double Talkin' Jive,Use Your Illusion I
Guns N' Roses,The Garden,Use Your Illusion I
Guns N' Roses,Don't Damn Me,Use Your Illusion I
Guns N' Roses,Dead Horse,Use Your Illusion I


## Задание 3
Для каждого набора (жанр, тип медиа) вывести среднюю цену по стоимости трека и общее количество, причем вывести только те наборы, для которых все треку стоят больше 1,5$.


In [7]:
%%sql
select Genre, Media_type, avg(Price) from
(select genres.Name as "Genre",
media_types.Name as "Media_type", tracks.UnitPrice as "Price"
from tracks
left join genres on tracks.GenreId = genres.GenreId
left join media_types on media_types.MediaTypeId = tracks.MediaTypeId
group by tracks.Name)
group by Genre
having min(Price) > 1.5


 * sqlite:///chinook.db
Done.


Genre,Media_type,avg(Price)
Comedy,Protected MPEG-4 video file,1.9899999999999995
Drama,Protected MPEG-4 video file,1.989999999999998
Sci Fi & Fantasy,Protected MPEG-4 video file,1.9900000000000004
Science Fiction,Protected MPEG-4 video file,1.9899999999999995
TV Shows,Protected MPEG-4 video file,1.990000000000001


## Задание 4
Вывести компании, сделавшие максимальное и минимальное число заказов.

In [8]:
%%sql
select Company, Count from 
(select Company, count(Company) as "Count" from
invoices
left join customers on customers.CustomerId = invoices.CustomerId
group by Company) 
where 
Count = (select max(Count) from 
(select count(Company) as "Count" from
invoices
left join customers on customers.CustomerId = invoices.CustomerId
group by Company) 
)
or Count = (select min(Count) from 
(select count(Company) as "Count" from
invoices
left join customers on customers.CustomerId = invoices.CustomerId
group by Company) 
)


 * sqlite:///chinook.db
Done.


Company,Count
None,0
Apple Inc.,7
Banco do Brasil S.A.,7
Embraer - Empresa Brasileira de Aeronáutica S.A.,7
Google Inc.,7
JetBrains s.r.o.,7
Microsoft Corporation,7
Riotur,7
Rogers Canada,7
Telus,7


## Задание 5
Для каждой компании вывести общее количестов песен, купленных по жанру поп-музыки

In [6]:
%%sql
select Company, sum(invoice_items.Quantity) from
invoices
left join invoice_items on invoices.InvoiceId = invoice_items.InvoiceId
left join customers on customers.CustomerId = invoices.CustomerId
left join tracks on tracks.TrackId = invoice_items.TrackId
left join genres on genres.GenreId = tracks.GenreId

where genres.Name LIKE 'Pop' and Company != "None"
group by Company

 * sqlite:///chinook.db
Done.


Company,sum(invoice_items.Quantity)
Embraer - Empresa Brasileira de Aeronáutica S.A.,2
JetBrains s.r.o.,4


## Задание 6
Вывести средний размер альбома в байтах.

In [87]:
%%sql
select avg(bytes) as "Average" from 
(select sum(Bytes) as "bytes" from
tracks
left join albums on tracks.AlbumId = albums.AlbumId
group by albums.Title)

 * sqlite:///chinook.db
Done.


Average
338288920.3170029
